In [1]:
import re
from datasets import load_dataset
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec
import numpy as np
import nltk
from datasets import load_dataset
import pickle

# Download required NLTK data
nltk.download('punkt')
nltk.download('stopwords')

/Users/juliankraus/Library/CloudStorage/OneDrive-Personal/Uni/Languages/Code/pythonProject/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/juliankraus/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/juliankraus/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# https://huggingface.co/datasets/legacy-datasets/wikipedia
dataset = load_dataset("wikipedia", "20220301.de")

Loading dataset shards:   0%|          | 0/18 [00:00<?, ?it/s]

In [8]:
num_examples = len(dataset['train'])
print(f"Number of examples: {num_examples}")


Number of examples: 2665357


# Cleaning

In [1]:
import re
import spacy
from datasets import load_dataset
from time import time
import os
import multiprocessing

# Load SpaCy model for German
nlp = spacy.load('de_core_news_sm')

# Load the dataset
dataset = load_dataset("wikipedia", "20220301.de")

# Preprocessing function
def preprocess_text(doc):
    stop_words = spacy.lang.de.stop_words.STOP_WORDS
    words = [token.text for token in doc if token.text not in stop_words and token.is_alpha]
    return words

# Prepare the texts for processing
texts = [article['text'] for article in dataset['train']]

# Brief cleaning to remove non-alphabetic characters
brief_cleaning = [re.sub(r'[^a-zA-ZäöüÄÖÜß\s]', '', text).lower() for text in texts]
with open('cleaned_texts.pkl', 'wb') as file:
    pickle.dump(brief_cleaning, file)


/Users/juliankraus/Library/CloudStorage/OneDrive-Personal/Uni/Languages/Code/pythonProject/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Loading dataset shards:   0%|          | 0/18 [00:00<?, ?it/s]

NameError: name 'pickle' is not defined

# Preprocessing

In [1]:
from keras.src.layers import LSTM, Dense
from keras import Sequential, Input
from keras.src.utils import pad_sequences
import pickle
import re
import spacy
from datasets import load_dataset
from time import time
import os
def preprocess_text(doc):
    stop_words = spacy.lang.de.stop_words.STOP_WORDS
    words = [token.text for token in doc if token.text not in stop_words and token.is_alpha]
    return words
nlp = spacy.load('de_core_news_sm', disable=['parser', 'ner'])
# Load the cleaned texts from the pickle file
with open('cleaned_texts.pkl', 'rb') as file:
    brief_cleaning = pickle.load(file)
# Measure time for processing
start_time = time()
num_cores = os.cpu_count()
print(f"Number of CPU cores: {num_cores}")

# Use spacy.pipe for concurrent processing
# Leave 1 or 2 cores free (for example, use num_cores - 1)
n_process = max(1, num_cores - 1)
# Use spacy.pipe for concurrent processing
processed_texts = [preprocess_text(doc) for doc in nlp.pipe(brief_cleaning, batch_size=100, n_process=10)]

# Measure and print the time taken for processing
end_time = time()
print(f"Time to clean up everything: {round((end_time - start_time) / 60, 2)} mins")

# Now `processed_texts` contains the cleaned and preprocessed texts
print(f"Processed {len(processed_texts)} articles.")

import pickle 
with open('database_preprocessed.pkl', 'wb') as file: 
    # A new file will be created 
    pickle.dump(processed_texts, file) 

/Users/juliankraus/Library/CloudStorage/OneDrive-Personal/Uni/Languages/Code/pythonProject/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Number of CPU cores: 16


/Users/juliankraus/Library/CloudStorage/OneDrive-Personal/Uni/Languages/Code/pythonProject/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/juliankraus/Library/CloudStorage/OneDrive-Personal/Uni/Languages/Code/pythonProject/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/juliankraus/Library/CloudStorage/OneDrive-Personal/Uni/Languages/Code/pythonProject/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/30

Time to clean up everything: 194.23 mins
Processed 2665357 articles.


# Word2Vec

In [ ]:
# Training the Word2Vec model
model = Word2Vec(
    sentences=processed_texts,
    vector_size=300,  # Dimensionality of the word vectors
    window=8,        # Maximum distance between the current and predicted word within a sentence
    min_count=5,     # Ignores all words with total frequency lower than this
    workers=10,       # Use these many worker threads to train the model
    sg=1,            # Training algorithm: 1 for skip-gram; otherwise CBOW
    epochs=10        # Number of iterations (epochs) over the corpus
)
model.save("word2vec.model")
import pickle 
with open('w2v.pkl', 'wb') as file: 
    pickle.dump(model, file) 

In [ ]:
import re
import spacy
from datasets import load_dataset
from time import time
import os
import multiprocessing

# Load SpaCy model for German
nlp = spacy.load('de_core_news_sm')
# Function to get word vectors for a given text
def preprocess_text(text):
    # Remove non-alphabetic characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    doc = nlp(text)
    stop_words = spacy.lang.de.stop_words.STOP_WORDS
    words = [token.text for token in doc if token.text not in stop_words and token.is_alpha]
    return words
def get_word2vec_embeddings(text, model, vector_size=300):
    tokens = preprocess_text(text)
    embeddings = np.zeros((len(tokens), vector_size))
    for i, token in enumerate(tokens):
        if token in model.wv:
            embeddings[i] = model.wv[token]
        else:
            embeddings[i] = np.zeros(vector_size)
    return embeddings

# Example text
text = "April is the fourth month of the year."
word2vec_embeddings = get_word2vec_embeddings(text, model)
print(word2vec_embeddings.shape)

# LSTM;

In [9]:
from keras.src.layers import LSTM, Dense
from keras import Sequential, Input
from keras.src.utils import pad_sequences
# Example dataset
texts = ["April is the fourth month of the year.", "May is the fifth month of the year."]
labels = [0, 1]

# Convert texts to embeddings
word2vec_embeddings = [get_word2vec_embeddings(text, model) for text in texts]
padded_embeddings = pad_sequences(word2vec_embeddings, padding='post', dtype='float32')

# LSTM model
model = Sequential([
    Input(shape=(padded_embeddings.shape[1], 100)),
    LSTM(128),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# Train the model
model.fit(padded_embeddings, labels, epochs=10)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 128)            │       117,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 117,377 (458.50 KB)

 Trainable params: 117,377 (458.50 KB)

 Non-trainable params: 0 (0.00 B)

ValueError: Unrecognized data type: x=[[[ 0.          0.          0.         ...  0.          0.
    0.        ]
  [-0.00992339 -0.00465845  0.06648266 ...  0.02166415 -0.20440356
   -0.1717809 ]
  [ 0.15835191 -0.24108157 -0.13057461 ...  0.08755513  0.17822938
    0.08964378]
  ...
  [ 0.2503722  -0.31800562  0.15515076 ...  0.22381121  0.07700533
    0.04427565]
  [ 0.15835191 -0.24108157 -0.13057461 ...  0.08755513  0.17822938
    0.08964378]
  [ 0.42727315  0.03994293 -0.02188285 ...  0.37655184  0.57135147
   -0.26862139]]

 [[ 0.          0.          0.         ...  0.          0.
    0.        ]
  [-0.00992339 -0.00465845  0.06648266 ...  0.02166415 -0.20440356
   -0.1717809 ]
  [ 0.15835191 -0.24108157 -0.13057461 ...  0.08755513  0.17822938
    0.08964378]
  ...
  [ 0.2503722  -0.31800562  0.15515076 ...  0.22381121  0.07700533
    0.04427565]
  [ 0.15835191 -0.24108157 -0.13057461 ...  0.08755513  0.17822938
    0.08964378]
  [ 0.42727315  0.03994293 -0.02188285 ...  0.37655